### import pandas
from matplotlib import rcParams
import requests
import json
import seaborn as sns
import itertools
from datetime import datetime
import matplotlib.pyplot as plt
import networkx as nx
import hashlib
import os
from operator import itemgetter

rcParams['figure.figsize'] = 11.7,8.27

def pretty_json(json_object):
    print(json.dumps(json_object, indent=2))

<img src="images/did_research_stop_during_covid.png" width="400">


.

.

.

.

.

..

.

..

.

.

.

..

.

.

.

.

.

.

.


<img src="images/khaby_lame.jpeg" width="200">

# No it didn't. 

(Betteridge's law of headlines)



## but... was there any visible change in the production of research output that we can see using the CORE API? 

### 1. Register for the CORE API https://core.ac.uk/services/api


In [1]:
# !pip install request
import requests
import json
api_key = "JAnjvcE7LDiB0aHeh8ruR3gUGFVW6qSI"
# with open ("apikey.txt", "r") as apikey_file:
#     api_key=apikey_file.readlines()[0].strip()
api_endpoint = "https://api.core.ac.uk/v3/"
def pretty_json(json_object): print(json.dumps(json_object, indent=2))

In [2]:
def get_entity(url_fragment):
    headers={"Authorization":"Bearer "+api_key}
    response = requests.get(api_endpoint + url_fragment, headers=headers)
    if response.status_code == 200:
        return response.json(), response.elapsed.total_seconds()
    else:
        print(f"Error code {response.status_code}, {response.content}")

In [3]:
data_provider, elapsed = get_entity("data-providers/1")
pretty_json(data_provider)

{
  "id": 1,
  "openDoarId": null,
  "name": "Aberdeen University Research",
  "email": "aura.deposit@abdn.ac.uk",
  "oaiPmhUrl": "https://aura.abdn.ac.uk/dspace-oai/request",
  "homepageUrl": null,
  "software": "dspace",
  "metadataFormat": "oai_dc",
  "createdDate": "2011-05-06T00:00:00+01:00",
  "location": {
    "countryCode": "GB",
    "latitude": 57.165,
    "longitude": -2.0991
  },
  "logo": "https://api.core.ac.uk/data-providers/1/logo",
  "type": "REPOSITORY",
  "rorId": "https://ror.org/0168zcn11",
  "institutionName": "Korea National University of Arts",
  "aliases": [
    "",
    "\ud55c\uad6d\uc608\uc220\uc885\ud569\ud559\uad50",
    "K-ARTS",
    "KNUA"
  ],
  "otherIdentifiers": {
    "ISNI": {
      "preferred": null,
      "all": [
        "0000 0001 2219 3809"
      ]
    },
    "OrgRef": {
      "preferred": null,
      "all": [
        "19680000"
      ]
    },
    "Wikidata": {
      "preferred": null,
      "all": [
        "Q489696"
      ]
    },
    "GRID": {

In [4]:
def query_api(url_fragment, query,limit=100):
    headers={"Authorization":"Bearer "+api_key}
    query = {"q":query, "limit":limit}
    response = requests.post(f"{api_endpoint}{url_fragment}",data = json.dumps(query), headers=headers)
    if response.status_code ==200:
        return response.json(), response.elapsed.total_seconds()
    else:
        print(f"Error code {response.status_code}, {response.content}")


In [5]:
results , elapsed = query_api("search/data-providers", "location.countryCode:gb")

pretty_json(results)

{
  "totalHits": 266,
  "limit": 100,
  "offset": 0,
  "results": [
    {
      "id": 2,
      "openDoarId": 1589,
      "name": "Abertay Research Portal",
      "email": "repository@abertay.ac.uk",
      "oaiPmhUrl": "https://rke.abertay.ac.uk/ws/oai",
      "homepageUrl": "http://repository.abertay.ac.uk",
      "software": "pure",
      "metadataFormat": "oai_dc",
      "createdDate": "2011-05-09T22:12:10+01:00",
      "location": {
        "countryCode": "GB",
        "latitude": 56.4599,
        "longitude": -2.9888
      },
      "logo": "https://api.core.ac.uk/data-providers/2/logo",
      "type": "REPOSITORY",
      "rorId": "https://ror.org/04mwwnx67",
      "institutionName": "Abertay University",
      "aliases": [],
      "otherIdentifiers": {
        "GRID": {
          "preferred": "grid.44361.34",
          "all": "grid.44361.34"
        },
        "ISNI": {
          "all": "0000 0001 0339 8665"
        },
        "FundRef": {
          "all": "100010040"
        },
   

In [6]:
def query_api(url_fragment, query,is_scroll=False, limit=100, scrollId=None):
    headers={"Authorization":"Bearer "+api_key}
    query = {"q":query, "limit":limit}
    if not is_scroll:
        response = requests.post(f"{api_endpoint}{url_fragment}",data = json.dumps(query), headers=headers)
    elif not scrollId:
        query["scroll"]="true"
        response = requests.post(f"{api_endpoint}{url_fragment}",data = json.dumps(query),headers=headers)
    else:
        query["scrollId"]=scrollId
        response = requests.post(f"{api_endpoint}{url_fragment}",data = json.dumps(query),headers=headers)
    if response.status_code ==200:
        return response.json(), response.elapsed.total_seconds()
    else:
        print(f"Error code {response.status_code}, {response.content}")

def scroll(search_url, query, extract_info_callback=None):
    allresults = []
    count = 0
    scrollId=None
    while True:
        result, elapsed =query_api(search_url, query, is_scroll=True, scrollId=scrollId)
        scrollId=result["scrollId"]
        totalhits = result["totalHits"]
        result_size = len(result["results"])
        if result_size==0:
            break
        for hit in result["results"]:
            if extract_info_callback:
              allresults.append(extract_info_callback(hit))
            else:
              allresults.append(hit)
        count+=result_size
        print(f"{count}/{totalhits} {elapsed}s")
    return allresults

In [7]:
def get_data_providers_id(hit):
    return {"id":hit["id"], "name":hit["name"]}

uk_data_providers_raw = scroll("search/data-providers", "location.countryCode:gb", get_data_providers_id)
uk_data_providers = pandas.DataFrame(uk_data_providers_raw)
uk_data_providers

100/266 4.906898s
200/266 2.972399s
266/266 5.120669s


NameError: name 'pandas' is not defined

In [ ]:
work, elapsed = get_entity("works/58886742")
pretty_json(work)

In [ ]:
work, elapsed = get_entity("works/10.3389/fmicb.2018.01845/full")
pretty_json(work)



In [ ]:
work, elapsed = get_entity("works/oai:strathprints.strath.ac.uk:4509")
pretty_json(work)

In [ ]:
work, elapsed = get_entity("works/core:277236443")
pretty_json(work)

In [ ]:
query = f"covid AND yearPublished>=2010 AND yearPublished<=2021"
results, elapsed = query_api("search/works", query, limit=1)
pretty_json(results)

In [ ]:
def aggregations(query, aggregation_fields,entity_type="works", limit=20, cache=True):
    headers={"Authorization":"Bearer "+api_key}

    query = {"q":query,"aggregations":aggregation_fields, "limit":limit}
    querystring = json.dumps(query).encode('utf-8')
    filename = f"cache/{hashlib.md5(querystring).hexdigest()}.csv"
    responseObject = {}
    if cache and os.path.exists(filename):
        with open (filename, "r") as cached:
            responseObject=json.loads(cached.readlines()[0].strip())
    else:
        response = requests.post(f"{api_endpoint}search/{entity_type}/aggregate",data = json.dumps(query), headers=headers)
        responseObject = response.json()
        with open (filename, "w") as cached:
            cached.write(json.dumps(responseObject))
        
    return responseObject




In [ ]:
query = f"covid AND yearPublished>=2010 AND yearPublished<=2021"
aggregation_response = aggregations(query, aggregation_fields=["yearPublished"],entity_type="works", limit=20)
pretty_json(aggregation_response)
year_data = aggregation_response["aggregations"]["yearPublished"]
years = pandas.DataFrame(list(year_data.items()), columns=["year", "records"]) 
years = years.sort_values("year", ascending=True)
ax = sns.barplot(x="year", y="records", data=years)

In [ ]:
def get_dataprovider_aggregation(query, year):
    connected_ids = " OR ".join(f"dataProviders:{id}" for id in uk_data_providers.id)
    query = f"{query} AND ({connected_ids}) AND yearPublished:{year}"
    aggregation_response = aggregations(query, aggregation_fields=["dataProviders"],entity_type="works")
    dp_data = aggregation_response["aggregations"]["dataProviders"]
    dps = pandas.DataFrame(list(dp_data.items()), columns=["dp_id", "records"]) 
    dps = dps[dps.dp_id.astype(int).isin(uk_data_providers.id)]
    dps["dp_id"]=dps["dp_id"].astype(int) 
    return dps.set_index("dp_id").join(uk_data_providers.set_index("id")).reset_index().sort_values("records", ascending=False)

In [ ]:
def plot_query(query, year, top_n=10):
    aggs = get_dataprovider_aggregation(query, year)
    ax = sns.barplot(x="records", y="name", data=aggs[:top_n])
    ax.set(xlabel="# records")
    ax.set(ylabel=None)
    plt.xticks(rotation=90)
    plt.title(f"\"{query}\" publications in {year}")
    plt.show()

plot_query("unprecedented times", 2018)
plot_query("unprecedented times", 2019)
plot_query("unprecedented times", 2020)
plot_query("unprecedented times", 2021)

In [ ]:
import itertools
russel_group = [119,286,27,83,33,39,504,14443,105,635,140,129,35,252,80,88,619,289,140,36,118,136, 42, 34]

target_dps =uk_data_providers.id.to_list()
def get_repo_name(url):
    id_repo = url.split("/")[-1]
    if uk_data_providers[uk_data_providers.id==int(id_repo)].any()["name"]:
            return uk_data_providers[uk_data_providers.id==int(id_repo)]["name"].values[0]
    return "Other data providers"

def get_repo_name_if_needed(dp_url):
    id_repo = dp_url.split("/")[-1]
    if int(id_repo) in target_dps:
        dp_name=get_repo_name(id_repo)
    else: 
        dp_name="Other data providers"
    return dp_name

def get_arcs(hit):
    results = []
    if len(hit["dataProviders"])==1:
        return results
    for dpA, dpB in itertools.combinations(hit["dataProviders"],2):
        dpA_name = get_repo_name_if_needed(dpA)
        dpB_name = get_repo_name_if_needed(dpB)        
        results.append({"source":dpA_name, "target":dpB_name, "edge":"co_deposit"})
    return results

def get_collaboration_network(query, cache=True):
    filename = f"cache/{hashlib.md5(query.encode('utf-8')).hexdigest()}.csv"
    edges_df = []
    if cache:
        if os.path.exists(filename):
            edges_df = pandas.read_csv(filename)
    if len(edges_df)==0:
        covid_works = scroll("search/works", query, get_arcs)
        works= []

        for c in covid_works:
            works.extend(c)

        edges_df = pandas.DataFrame(works)
        edges_df.to_csv(filename)
        
        
    edges_df = edges_df[edges_df.target!="Other data providers"][edges_df.source!="Other data providers"].groupby(['source', "target"]).count().reset_index()

    G=nx.from_pandas_edgelist(edges_df,  "source", "target", edge_attr=True, create_using=nx.Graph())
    plt.figure(figsize=(40,40))
    M = G.number_of_edges()
    edge_colors = range(2, M + 2)
    cmap = sns.color_palette("viridis", as_cmap=True)
    pos = nx.circular_layout(G)
    widths = 15 * (edges_df["edge"]/edges_df["edge"].max()) +1
    nodes = nx.draw_networkx_nodes(G, pos, node_color="indigo" )
    edges = nx.draw_networkx_edges(
        G,
        pos,
        edge_color=edge_colors,
        edge_cmap=cmap,
        width=widths,
        
    )
    label_options = {"fc": "white"}
    nx.draw_networkx_labels(G, pos, font_size=14, bbox=label_options)
    plt.show()
    #return edges_df.count()

In [ ]:
def get_ego_network(query, cache=True):
    filename = f"cache/{hashlib.md5(query.encode('utf-8')).hexdigest()}.csv"
    edges_df = []
    if cache:
        if os.path.exists(filename):
            edges_df = pandas.read_csv(filename)
    if len(edges_df)==0:
        covid_works = scroll("search/works", query, get_arcs)
        works= []

        for c in covid_works:
            works.extend(c)

        edges_df = pandas.DataFrame(works)
        edges_df.to_csv(filename)
        
        
    edges_df = edges_df[edges_df.target!="Other data providers"][edges_df.source!="Other data providers"].groupby(['source', "target"]).count().reset_index()    
    
    
    G=nx.from_pandas_edgelist(edges_df,  "source", "target", edge_attr=True, create_using=nx.Graph())
    plt.figure(figsize=(40,40))
    M = G.number_of_edges()
    edge_colors = range(2, M + 2)
    cmap = sns.color_palette("viridis", as_cmap=True)
    pos = nx.circular_layout(G)
    widths = 15 * (edges_df["edge"]/edges_df["edge"].max()) +1
    # Create a BA model graph - use seed for reproducibility
    seed = 20532

    # find node with largest degree
    node_and_degree = G.degree()
    (largest_hub, degree) = sorted(node_and_degree, key=itemgetter(1))[-1]

    # Create ego graph of main hub
    hub_ego = nx.ego_graph(G, largest_hub)

    # Draw graph
    pos = nx.spring_layout(hub_ego, seed=seed)  # Seed layout for reproducibility
    nx.draw(hub_ego, pos, node_color="b", node_size=50, with_labels=False)

    # Draw ego as large and red
    options = {"node_size": 300, "node_color": "r"}
    label_options = {"fc": "white"}
    nx.draw_networkx_nodes(hub_ego, pos, nodelist=[largest_hub], **options)
    nx.draw_networkx_labels(hub_ego, pos, font_size=24, bbox=label_options)
    edges = nx.draw_networkx_edges(
        hub_ego,
        pos,
        edge_color=edge_colors,
        edge_cmap=cmap,
        width=widths,
        
    )
    plt.show()
    #return edges_df.count()

In [ ]:
query = f"covid AND yearPublished:2021 AND dataProviders:140"
get_ego_network(query)

In [ ]:
ids_to_focus = " OR ".join(f"dataProviders:{id}" for id in russel_group)
query = f"covid AND yearPublished:2021 AND ({ids_to_focus})"
get_collaboration_network(query, cache=True)